<a href="https://colab.research.google.com/github/connecttopawan/Machine_Learning/blob/main/Grade_Prediction_Linear_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***Grade Prediction - Linear Regression.ipynb***

In [ ]:
#Connecting Drive to Colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Setting up PySpark in Colab
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [ ]:
#Installing pyspark
!pip install -q findspark

In [ ]:
#Setup environment variable
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [ ]:
#Start spark session
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Linear Regression").getOrCreate()
#spark = session.builder.master("local[1]").appName("Linear Regression").getOrCreate()


In [ ]:
# uploading files from local to colab location
from google.colab import files
files.upload()

Saving Student_Grades_Data.csv to Student_Grades_Data.csv


{'Student_Grades_Data.csv': b'Time_to_Study,Grades\r\n1,1.5\r\n5,2.7\r\n7,3.1\r\n3,2.1\r\n2,1.8\r\n9,3.9\r\n6,2.9\r\n12,4.5\r\n11,4.3\r\n2,1.8\r\n4,2.4\r\n8,3.5\r\n13,4.8\r\n9,3.9\r\n14,5\r\n10,4.1\r\n6,2.9\r\n12,4.5\r\n1,1.5\r\n4,2.4\r\n14,5\r\n10,4.1\r\n11,4.3\r\n4,2.4\r\n5,2.7\r\n8,3.5\r\n1,1.5\r\n2,1.8\r\n3,2.1\r\n7,3.1\r\n8,3.5\r\n14,5\r\n7,3.1\r\n8,3.5\r\n1,1.5\r\n2,1.8\r\n3,2.1\r\n4,2.4\r\n5,2.7\r\n6,2.9\r\n7,3.1\r\n8,3.5\r\n9,3.9\r\n10,4.1\r\n11,4.3\r\n12,4.5\r\n13,4.8\r\n14,5\r\n8,3.5\r\n2,1.8\r\n'}

In [ ]:
# Reading files through spark session
data=spark.read.format('csv').option('header','true').option('inferSchema','true').load('Student_Grades_Data.csv')
data.printSchema()
data.show(5)

root
 |-- Time_to_Study: integer (nullable = true)
 |-- Grades: double (nullable = true)

+-------------+------+
|Time_to_Study|Grades|
+-------------+------+
|            1|   1.5|
|            5|   2.7|
|            7|   3.1|
|            3|   2.1|
|            2|   1.8|
+-------------+------+
only showing top 5 rows



In [ ]:
# Creating feature array
features=data.columns[0:1]
from pyspark.ml.feature import VectorAssembler
vector_asmblr=VectorAssembler(inputCols=features, outputCol="Features")

VectorAssembler_82be4613f27f


In [ ]:
# Adding feature columns to data
featured_data=vector_asmblr.transform(data)
featured_data.show()

+-------------+------+--------+
|Time_to_Study|Grades|Features|
+-------------+------+--------+
|            1|   1.5|   [1.0]|
|            5|   2.7|   [5.0]|
|            7|   3.1|   [7.0]|
|            3|   2.1|   [3.0]|
|            2|   1.8|   [2.0]|
|            9|   3.9|   [9.0]|
|            6|   2.9|   [6.0]|
|           12|   4.5|  [12.0]|
|           11|   4.3|  [11.0]|
|            2|   1.8|   [2.0]|
|            4|   2.4|   [4.0]|
|            8|   3.5|   [8.0]|
|           13|   4.8|  [13.0]|
|            9|   3.9|   [9.0]|
|           14|   5.0|  [14.0]|
|           10|   4.1|  [10.0]|
|            6|   2.9|   [6.0]|
|           12|   4.5|  [12.0]|
|            1|   1.5|   [1.0]|
|            4|   2.4|   [4.0]|
+-------------+------+--------+
only showing top 20 rows



In [ ]:
# Selecting only feature and label columns
final_data=featured_data.select('Features','Grades')

# Splitting train and test data
train_dataset, test_dataset=final_data.randomSplit([0.7,0.3]) 
train_dataset.describe().show()
test_dataset.describe().show()

+-------+------------------+
|summary|            Grades|
+-------+------------------+
|  count|                39|
|   mean|3.2230769230769227|
| stddev|1.0817776643006443|
|    min|               1.5|
|    max|               5.0|
+-------+------------------+

+-------+------------------+
|summary|            Grades|
+-------+------------------+
|  count|                11|
|   mean|3.2181818181818187|
| stddev| 1.238400728212142|
|    min|               1.5|
|    max|               5.0|
+-------+------------------+



In [ ]:
# Creating Linear Regression object
from pyspark.ml.regression import LinearRegression
linear_reg=LinearRegression(featuresCol="Features", labelCol="Grades")

In [36]:
# Training model
lg_model=linear_reg.fit(train_dataset)

# Predicting model
pred_model=lg_model.evaluate(test_dataset)

predicted_data=pred_model.predictions
predicted_data.show()

+--------+------+------------------+
|Features|Grades|        prediction|
+--------+------+------------------+
|   [1.0]|   1.5|1.5533794823286597|
|   [2.0]|   1.8|1.8258405709863261|
|   [3.0]|   2.1| 2.098301659643992|
|   [4.0]|   2.4| 2.370762748301659|
|   [4.0]|   2.4| 2.370762748301659|
|   [8.0]|   3.5| 3.460607102932324|
|   [8.0]|   3.5| 3.460607102932324|
|  [10.0]|   4.1| 4.005529280247657|
|  [11.0]|   4.3| 4.277990368905323|
|  [13.0]|   4.8| 4.822912546220655|
|  [14.0]|   5.0| 5.095373634878322|
+--------+------+------------------+



In [35]:
# Finding Coefficient and intercept of model
coefficient=lg_model.coefficients
intercept=lg_model.intercept
print("Coefficient : ",coefficient,"\nIntercept : ", intercept)


Coefficient :  [0.27246108865766633] 
Intercept :  1.2809183936709934


In [43]:
# Evaluating the model
from pyspark.ml.evaluation import RegressionEvaluator
evaluation=RegressionEvaluator(labelCol="Grades", predictionCol='prediction')

# Root Mean Square Error
rmse=evaluation.evaluate(predicted_data,{evaluation.metricName: "rmse"})

# Mean Square Error
mse=evaluation.evaluate(predicted_data,{evaluation.metricName : "mse"})

#Mean Absolute Error
mae=evaluation.evaluate(predicted_data,{evaluation.metricName : "mae"})

# Coefficient of determination (r2)
r2=evaluation.evaluate(predicted_data,{evaluation.metricName : "r2"})
r2


0.9982157779080147